In [1]:
import carla

In [2]:
# spawn firetruck at start_point
client = carla.Client('10.8.179.139', 2000)

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter("firetruck*")
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [3]:
# get the car's position on the map
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471968, z=-0.001595), Rotation(pitch=0.000020, yaw=0.159199, roll=0.000936))


In [4]:
# initial spawn point is the same - just 0.6m higher off the ground
print(start_point)

Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [5]:
# send vehicle off
vehicle.set_autopilot(True)

In [8]:
# get actual position from the car moving
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-23.173193, y=13.119796, z=-0.003491), Rotation(pitch=-0.021877, yaw=-179.830887, roll=-0.000244))


In [9]:
# now look at the map
town_map = world.get_map()

In [8]:
type(town_map)

carla.libcarla.Map

In [9]:
print(town_map)

Map(name=Carla/Maps/Town10HD_Opt)


In [10]:
roads = town_map.get_topology()

In [11]:
# topology is pairs of waypoints defining all roads - tuples of those
print(roads)

[(<carla.libcarla.Waypoint object at 0x7f55680f70f0>, <carla.libcarla.Waypoint object at 0x7f55680f7150>), (<carla.libcarla.Waypoint object at 0x7f55680f71b0>, <carla.libcarla.Waypoint object at 0x7f55680f7210>), (<carla.libcarla.Waypoint object at 0x7f55680f7270>, <carla.libcarla.Waypoint object at 0x7f55680f72d0>), (<carla.libcarla.Waypoint object at 0x7f55680f7330>, <carla.libcarla.Waypoint object at 0x7f55680f7390>), (<carla.libcarla.Waypoint object at 0x7f55680f73f0>, <carla.libcarla.Waypoint object at 0x7f55680f7450>), (<carla.libcarla.Waypoint object at 0x7f55680f74b0>, <carla.libcarla.Waypoint object at 0x7f55680f7510>), (<carla.libcarla.Waypoint object at 0x7f55680f7570>, <carla.libcarla.Waypoint object at 0x7f55680f75d0>), (<carla.libcarla.Waypoint object at 0x7f55680f7630>, <carla.libcarla.Waypoint object at 0x7f55680f7690>), (<carla.libcarla.Waypoint object at 0x7f55680f76f0>, <carla.libcarla.Waypoint object at 0x7f55680f7750>), (<carla.libcarla.Waypoint object at 0x7f55680

In [12]:
# have a look at a waypoint - it is transform wrapped as a waypoint
print(roads[0][0])

Waypoint(Transform(Location(x=-27.017662, y=66.214005, z=0.000000), Rotation(pitch=0.000000, yaw=-179.926727, roll=0.000000)))


In [13]:
len(roads)

200

### GlobalRoutePlanner

In [7]:
import sys
sys.path.append('/opt/carla-simulator/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [11]:
# using the code to plan the route and then draw it in the simulator
# town10HD_map = world.get_map()

sampling_resolution = 2

grp = GlobalRoutePlanner(town_map, sampling_resolution)

point_a = carla.Location(x=-64.644844, y=24.471010, z=0.600000)
point_b = carla.Location(x=-113.903656, y=14.422489, z=-0.003719)

route = grp.trace_route(point_a, point_b) # there are other functions can be used

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
                           color=carla.Color(r=0, g=0, b=255), life_time=120.0,
                           persistent_lines=True)


In [12]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
    
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

### now we define 2 cars


In [14]:
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

# start first car in already defined start point
truck = world.try_spawn_actor(truck_bp[0], start_point)

In [15]:
# twak spectator position to watch the show

spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4), # automatically move 
                                                                      carla.Rotation(yaw = start_point.rotation.yaw -155))

spectator.set_transform(spectator_pos)

In [16]:
# drop the Mini the sky -  watch what happens after

# spawn it first somewhere else
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10])

mini_pos = carla.Transform(start_point.location + carla.Location(x=-4, z=10), carla.Rotation(yaw=start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)